The UCSC [Table Browser](https://genome.ucsc.edu/cgi-bin/hgTables) allows a nice interface for downloading the pairwise alignments between two genomes in table format. While UCSC offers this excellent feature, there are fewer pairwise alignments available than those in Ensembl. However, trying to get a useful format of the pairwise alignments from Ensembl is more difficult. You can directly download the multiple alignment files in MAF format, but I want something like the UCSC table browser. It turns out Ensembls public mysql server can be used to get any of the data hosted on ensembl databases. Here I show how to grab the synteny blocks between human and mouse using [pymysql](https://github.com/PyMySQL/PyMySQL) and read the resulting table into a pandas dataframe. The Ensemble compara database has a good MySql [schema documentation](https://ensembl-compara.readthedocs.io/en/master/schema.html), this is helpful when figuring out how to get around the relational mess.

First download pymysql with a quick `pip install pymysql`, and make sure you have `pandas` in your current environment.

In [1]:
import pandas as pd
import pymysql
%matplotlib inline

In [19]:
# connect to the ensembl MySql db
connection = pymysql.connect(host='ensembldb.ensembl.org',
                             user='anonymous',
                             port=5306,
                             password="",
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)


Conveniently, pandas has a function `read_sql` allowing you to perform query `q` on connection `conn`, and print the results to a pandas dataframe. This is super useful if you plan to further manipulate the data you gather, and maybe do some quick exploratory data analysis.

In [20]:
databases = pd.read_sql("show databases;", connection)
databases.head()

,Database
0,information_schema
1,PERCONA_SCHEMA
2,acanthochromis_polyacanthus_core_94_1
3,acanthochromis_polyacanthus_core_95_1
4,acanthochromis_polyacanthus_core_96_1


There are quite a few databases we have access to, but I'm interested in the one that shows syntenic regions between to species, this will be in the compara database. We can search for that below.

In [12]:
databases[databases.Database.str.match(".+compara.+")].tail()

,Database
2459,ensembl_compara_92
2460,ensembl_compara_93
2461,ensembl_compara_94
2462,ensembl_compara_95
2463,ensembl_compara_96


Connect to the most recent version, and start building a query. I had to brush up on SQL syntax, and take a look at the examples in the [Compara Schema Docs](https://uswest.ensembl.org/info/docs/api/compara/compara_schema.html#species_set_header).

In [24]:
compara_96 = pymysql.connect(host='ensembldb.ensembl.org',
                             user='anonymous',
                             db="ensembl_compara_96",
                             password="",
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

It's not pretty, but here's what I ended up with to get a table containing syntenic blocks between human and mouse:

In [25]:
qAll = '\
SELECT t1.species_name as sname, t1.name as schr, t1.dnafrag_start as sstart, t1.dnafrag_end as send, t1.dnafrag_strand as sstrand, \
            t2.species_name as qname, t2.name as qchr, t2.dnafrag_start as qstart, t2.dnafrag_end as qend, t2.dnafrag_strand as qstrand, t1.synteny_region_id, t2.synteny_region_id \
     FROM (SELECT genome_db.name AS species_name, \
                 dnafrag.name, dnafrag_start, \
                 dnafrag_end, dnafrag_strand, \
                 synteny_region_id \
      FROM dnafrag_region \
      JOIN dnafrag USING (dnafrag_id) \
      JOIN genome_db USING (genome_db_id) \
      WHERE genome_db.name IN ("mus_musculus") \
      ORDER BY synteny_region_id) as t1, \
     (SELECT genome_db.name AS species_name, \
                dnafrag.name, dnafrag_start, \
                dnafrag_end, dnafrag_strand, \
                synteny_region_id \
      FROM dnafrag_region \
      JOIN dnafrag USING (dnafrag_id) \
      JOIN genome_db USING (genome_db_id) \
      WHERE genome_db.name IN ("homo_sapiens") \
      ORDER BY synteny_region_id) as t2 \
WHERE t1.synteny_region_id=t2.synteny_region_id \
ORDER BY schr;'

mouse_human_syn = pd.read_sql(qAll, compara_96)
mouse_human_syn.head()

,sname,schr,sstart,send,sstrand,qname,qchr,qstart,qend,qstrand,synteny_region_id,synteny_region_id
0,mus_musculus,1,3007339,9473382,-1,homo_sapiens,8,49798969,55650088,1,51768,51768
1,mus_musculus,1,43917041,44220159,1,homo_sapiens,13,102585255,102881564,1,51528,51528
2,mus_musculus,1,134780249,134938733,1,homo_sapiens,Y,26212371,26397883,1,51816,51816
3,mus_musculus,1,44337306,46007489,1,homo_sapiens,2,188142550,189639740,1,51635,51635
4,mus_musculus,1,9522893,17933852,1,homo_sapiens,8,66403591,75214401,1,51772,51772


In [26]:
mouse_human_syn.shape[0]

363

Cool, Ensembl has 363 syntenic blocks between human and mouse. These are derived from the full `lastz` alignment net, which chains the most contiguous blocks to filter out some noise. Because there are more lastz alignments, than synteny's in Ensembl, you can apply a similar technique as I did above using the [Genomic Alignments](https://ensembl-compara.readthedocs.io/en/master/schema.html#genomic-alignments) tables. There will be many more alignment regions, much of the noise can be filtered out using the `score` attribute which is the lastz alignment score, a higher score resulting in fewer, better quality alignments. 

I am excited about this way to programmatically access ensembl data through Python and jupyter notebooks, as it means I don't have to use the Perl API (which is a huge set of perl modules), and I can perform analysis right in the same notebook the data was fetched from. It feels more transparent for doing this type of exploratory data analysis, and really allows some flexibility that's not allowed in the UI, or the Rest API. I hope someone else finds this useful. I like this method because it requires relatively very few lines of code and only two dependencies. 